In [8]:
pip install numba


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from itertools import product
import numpy as np
from scipy.sparse import identity
from scipy.sparse.linalg import expm
import matplotlib.pyplot as plt
from numba import njit, prange

@njit
def generate_basis(n):
    basis = []
    L = n + 1  # Length of the spin chain plus one for periodic boundary conditions
    for i in range(2 ** L):
        state = np.array([(i >> j) & 1 for j in range(L)], dtype=np.int32)
        valid = True
        for j in range(L):
            if state[j] == 1:
                j1p = (j + 1) % L
                jp = j
                if state[j1p] == 1 and state[jp] == 1:
                    valid = False
                    break
        if valid:
            basis.append(state)
    return basis

#@njit
def construct_hamiltonian(bases, mu_omega):
    d = len(bases)
    L = lent+1
    hmat = np.zeros((d, d), dtype=np.float64)
    
    for i in range(d):
        state = bases[i]
        nj = 0
        for j in range(L):
            n_s = state[j]
            nj += ((-1)**j * n_s) + ((1 - (-1)**j) / 2)
            
            j1p = (j + 1) % L
            jp = j
            j0p = (j - 1) if j > 0 else L - 1
            
            if state[j1p] == 0 and state[jp] == 1 and state[j0p] == 0:
                new_state = state.copy()
                new_state[jp] = 0
                for k in range(d):
                    if np.array_equal(bases[k], new_state):
                        hmat[i, k] = 1
                        break
                
            if state[j1p] == 0 and state[jp] == 0 and state[j0p] == 0:
                new_state = state.copy()
                new_state[jp] = 1
                for k in range(d):
                    if np.array_equal(bases[k], new_state):
                        hmat[i, k] = 1
                        break
                
        hmat[i, i] += mu_omega * nj

    return hmat

@njit(parallel=True)
def time_evolution(bases, hmat, start_state, time_steps):
    d = len(bases)
    lent = bases.shape[1]
    ma = np.zeros(time_steps, dtype=np.float64)
    
    for n in prange(time_steps):
        tlt = -1j * hmat * n
        ut = expm(tlt)
        rho0 = identity(d).toarray()
        rhot = ut @ rho0
        
        M = 0
        for i in range(d):
            state = bases[i]
            prob = rhot[i, start_state].real
            if prob != 0:
                nj = sum(((-1)**j * state[j] + ((1 - (-1)**j) / 2)) for j in range(lent))
                M += prob * nj
        ma[n] = M / (lent * time_steps)
    
    return ma

lent = 5
bases = generate_basis(lent)
mu_omega = 0
time_steps = 100
start_state = 3

hmat = construct_hamiltonian(bases, mu_omega)
ma = time_evolution(bases, hmat, start_state, time_steps)

plt.plot(ma)
plt.xlabel('Time Steps')
plt.ylabel('Observable M')
plt.title('Time Evolution of Observable M')
plt.show()


UnsupportedError: Failed in nopython mode pipeline (step: inline calls to locally defined closures)
The use of yield in a closure is unsupported.

File "../../../../tmp/ipykernel_163671/263875149.py", line 80:
<source missing, REPL/exec in use?>


In [4]:
from itertools import product
import numpy as np
from scipy.sparse import identity
from scipy.sparse.linalg import expm
import matplotlib.pyplot as plt
from numba import njit, prange

@njit
def generate_basis(L):
    num_states = 2 ** L
    basis = []

    for i in range(num_states):
        state = np.array([(i >> j) & 1 for j in range(L)], dtype=np.int32)
        valid = True
        for j in range(L):
            if state[j] == 1:
                j1p = (j + 1) % L
                jp = j
                if state[j1p] == 1 and state[jp] == 1:
                    valid = False
                    break
        if valid:
            basis.append(state)
    
    return np.array(basis)

@njit
def construct_hamiltonian(bases, mu_omega):
    d = len(bases)
    L = bases.shape[1]
    hmat = np.zeros((d, d), dtype=np.float64)
    
    for i in range(d):
        state = bases[i]
        nj = 0
        for j in range(L):
            n_s = state[j]
            nj += ((-1) ** j * n_s) + ((1 - (-1) ** j) / 2)
            
            j1p = (j + 1) % L
            jp = j
            j0p = (j - 1) if j > 0 else L - 1
            
            if state[j1p] == 0 and state[jp] == 1 and state[j0p] == 0:
                new_state = state.copy()
                new_state[jp] = 0
                for k in range(d):
                    if np.array_equal(bases[k], new_state):
                        hmat[i, k] = 1
                        break
                
            if state[j1p] == 0 and state[jp] == 0 and state[j0p] == 0:
                new_state = state.copy()
                new_state[jp] = 1
                for k in range(d):
                    if np.array_equal(bases[k], new_state):
                        hmat[i, k] = 1
                        break
                
        hmat[i, i] += mu_omega * nj

    return hmat

@njit(parallel=True)
def time_evolution(bases, hmat, start_state, time_steps):
    d = len(bases)
    L = bases.shape[1]
    ma = np.zeros(time_steps, dtype=np.float64)
    
    for n in prange(time_steps):
        tlt = -1j * hmat * n
        ut = expm(tlt)
        rho0 = identity(d).toarray()
        rhot = ut @ rho0
        
        M = 0
        for i in range(d):
            state = bases[i]
            prob = rhot[i, start_state].real
            if prob != 0:
                nj = sum(((-1) ** j * state[j] + ((1 - (-1) ** j) / 2)) for j in range(L))
                M += prob * nj
        ma[n] = M / (L * time_steps)
    
    return ma

def simulate(lent=16, mu_omega=0, time_steps=100, start_state=3):
    bases = generate_basis(lent)
    hmat = construct_hamiltonian(bases, mu_omega)
    ma = time_evolution(bases, hmat, start_state, time_steps)
    
    plt.plot(ma)
    plt.xlabel('Time Steps')
    plt.ylabel('Observable M')
    plt.title('Time Evolution of Observable M')
    plt.show()

# Parameters
lent = 16
mu_omega = 0
time_steps = 100
start_state = 3

simulate(lent, mu_omega, time_steps, start_state)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(list(array(int32, 1d, C))<iv=None>)
 
There are 2 candidate implementations:
   - Of which 2 did not match due to:
   Overload in function 'impl_np_array': File: numba/np/arrayobj.py: Line 5432.
     With argument(s): '(list(array(int32, 1d, C))<iv=None>)':
    Rejected as the implementation raised a specific error:
      TypingError: Failed in nopython mode pipeline (step: nopython frontend)
    No implementation of function Function(<intrinsic np_array>) found for signature:
     
     >>> np_array(list(array(int32, 1d, C))<iv=None>, none)
     
    There are 2 candidate implementations:
          - Of which 2 did not match due to:
          Intrinsic in function 'np_array': File: numba/np/arrayobj.py: Line 5406.
            With argument(s): '(list(array(int32, 1d, C))<iv=None>, none)':
           Rejected as the 

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

@njit
def generate_basis(L):
    num_states = 2 ** L
    basis = np.zeros((num_states, L), dtype=np.int32)
    count = 0
    
    for i in range(num_states):
        state = np.zeros(L, dtype=np.int32)
        for j in range(L):
            state[j] = (i >> j) & 1
        valid = True
        for j in range(L):
            if state[j] == 1:
                j1p = (j + 1) % L
                jp = j
                if state[j1p] == 1 and state[jp] == 1:
                    valid = False
                    break
        if valid:
            basis[count] = state
            count += 1

    return basis[:count]

@njit
def construct_hamiltonian(bases, mu_omega):
    d = len(bases)
    L = bases.shape[1]
    hmat = np.zeros((d, d), dtype=np.float64)
    
    for i in range(d):
        state = bases[i]
        nj = 0
        for j in range(L):
            n_s = state[j]
            nj += ((-1)**j * n_s) + ((1 - (-1)**j) / 2)
            
            j1p = (j + 1) % L
            jp = j
            j0p = (j - 1) if j > 0 else L - 1
            
            if state[j1p] == 0 and state[jp] == 1 and state[j0p] == 0:
                new_state = state.copy()
                new_state[jp] = 0
                for k in range(d):
                    if np.array_equal(bases[k], new_state):
                        hmat[i, k] = 1
                        break
                
            if state[j1p] == 0 and state[jp] == 0 and state[j0p] == 0:
                new_state = state.copy()
                new_state[jp] = 1
                for k in range(d):
                    if np.array_equal(bases[k], new_state):
                        hmat[i, k] = 1
                        break
                
        hmat[i, i] += mu_omega * nj

    return hmat

@njit
def matrix_exponential(mat, t):
    mat_exp = np.eye(mat.shape[0], dtype=np.complex128)
    mat_term = np.eye(mat.shape[0], dtype=np.complex128)
    factorial = 1
    
    for n in range(1, 50):  # Arbitrary choice to ensure convergence
        factorial *= n
        mat_term = mat_term @ (mat * (t / n))
        mat_exp += mat_term / factorial

    return mat_exp

@njit(parallel=True)
def time_evolution(bases, hmat, start_state, time_steps):
    d = len(bases)
    L = bases.shape[1]
    ma = np.zeros(time_steps, dtype=np.float64)
    
    for n in prange(time_steps):
        ut = matrix_exponential(-1j * hmat, n)
        rho0 = np.eye(d)
        rhot = ut @ rho0
        
        M = 0
        for i in range(d):
            state = bases[i]
            prob = rhot[i, start_state].real
            if prob != 0:
                nj = sum(((-1)**j * state[j] + ((1 - (-1)**j) / 2)) for j in range(L))
                M += prob * nj
        ma[n] = M / (L * time_steps)
    
    return ma

def simulate(lent=16, mu_omega=0, time_steps=100, start_state=3):
    bases = generate_basis(lent)
    hmat = construct_hamiltonian(bases, mu_omega)
    ma = time_evolution(bases, hmat, start_state, time_steps)
    
    plt.plot(ma)
    plt.xlabel('Time Steps')
    plt.ylabel('Observable M')
    plt.title('Time Evolution of Observable M')
    plt.show()

# Parameters
lent = 16
mu_omega = 0
time_steps = 100
start_state = 3

simulate(lent, mu_omega, time_steps, start_state)


UnsupportedError: Failed in nopython mode pipeline (step: inline calls to locally defined closures)
The use of yield in a closure is unsupported.

File "../../../../tmp/ipykernel_163671/3148640413.py", line 95:
<source missing, REPL/exec in use?>
